## Run the below import cell first

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sqlite3
from sklearn.preprocessing import MinMaxScaler

# The four cells below this one will create the same type of DF used in the tutorial. Next step, killing my CPU or GPU

**NOTE**: I think eventually I'd like to add other information in here, such as the sector IDs and such for the symbols.

In [45]:
# One

conn = sqlite3.connect('stockPrediction.db')

query = "SELECT r.stock_symbol, l.price_datetime, l.open_price, l.high_price, l.low_price, l.close_price, l.volume, l.dividends, l.stock_splits FROM price_history l INNER JOIN stock r ON r.stock_id = l.stock_id;"

symbols = conn.execute('SELECT stock_symbol FROM stock')
symbols = symbols.fetchall()
symbols = [i[0] for i in symbols]
symbols = [i for i in symbols if i not in symbols]

# columns = pd.MultiIndex.from_product([[i for i in symbols], ['open_price', 'high_price', 'low_price', 'close_price']], names=['symbol', 'type'])

df = pd.read_sql(query, conn, index_col=['stock_symbol', 'price_datetime'])

In [38]:
df.shape

(1896354, 7)

In [33]:
df = df.reset_index()
df = df[df['price_datetime'].str.contains("00:00|15:00|30:00|45:00")==True]
# df.price_datetime.unique()

In [34]:
df.shape

(1892282, 9)

In [54]:
for i in df.price_datetime.unique():
    print(i)

T10:45:00.000000000
2021-03-10T12:45:00.000000000
2021-03-10T13:00:00.000000000
2021-03-10T13:15:00.000000000
2021-03-10T14:15:00.000000000
2021-03-10T14:30:00.000000000
2021-03-12T12:00:00.000000000
2021-03-15T10:30:00.000000000
2021-03-15T11:30:00.000000000
2021-03-15T13:00:00.000000000
2021-03-15T14:15:00.000000000
2021-03-15T14:30:00.000000000
2021-03-17T09:30:00.000000000
2021-03-17T10:30:00.000000000
2021-03-17T11:00:00.000000000
2021-03-17T11:15:00.000000000
2021-03-17T11:30:00.000000000
2021-03-17T12:15:00.000000000
2021-03-17T13:45:00.000000000
2021-03-17T14:15:00.000000000
2021-03-17T15:30:00.000000000
2021-03-18T12:00:00.000000000
2021-03-18T13:00:00.000000000
2021-03-18T14:15:00.000000000
2021-03-22T12:45:00.000000000
2021-03-23T11:30:00.000000000
2021-03-23T11:45:00.000000000
2021-03-26T11:00:00.000000000
2021-03-26T15:00:00.000000000
2021-03-29T11:00:00.000000000
2021-03-30T12:30:00.000000000
2021-03-30T12:45:00.000000000
2021-03-30T13:30:00.000000000
2021-03-30T15:30:00.

In [46]:
df = df.reset_index()
df['price_datetime'] = pd.to_datetime(df['price_datetime'])

In [53]:
df = df.loc[df['price_datetime'].dt.second == 0]

In [ ]:
type(df['price_datetime'][])

In [ ]:
for i, row in df.iterrows():
    # print(i)
    if "00:00" in i[1]:
        pass
    elif "15:00" in i[1]:
        pass
    elif "30:00" in i[1]:
        pass
    elif "45:00" in i[1]:
        pass
    else:
        try:
            df.drop(index=(i[0], i[1]))
        except:
            print("Unable to drop", i)

In [ ]:
df.reset_index()
df.price_datetime.unique()

In [ ]:
# Two

# remove stocks w/ less than 1000 data points
s = df.groupby(level=0).size()
s = s.where(s > 1000).dropna()
dfs = s.to_frame()
dfs = dfs.reset_index()
df = df.reset_index()
dfs

In [ ]:
# Three
df = pd.merge(df, dfs, how='inner', left_on='stock_symbol', right_on='stock_symbol', left_index=True)
df = df.drop_duplicates()
df

In [ ]:
# Four

# df = df.drop_duplicates(subset=['price_datetime'])
df = df.set_index(['price_datetime', 'stock_symbol']).unstack(['stock_symbol'])
df

## Test-train split below
Uses close prices only. Will try a future version that uses all available data.

In [ ]:
# first cell for test-train split
data = df[['close_price']]
data = data.reset_index()

# I am doing this differently. I am using 80% as training data, 20% as test
data_size = len(data)
training_data = data.iloc[:(int(data_size * .8))]
test_data = data.iloc[(int(data_size * .8)):]

In [ ]:
# second cell for test-train split
# adding the price_datetime index back
training_data = training_data.set_index('price_datetime')
test_data = test_data.set_index('price_datetime')

In [ ]:
# testing the sets w/ matplotlib
plt.figure(figsize=(14,4))
plt.plot(training_data.close_price['BLPH'])
plt.plot(test_data.close_price['BLPH'])
plt.ylabel('Close Price')
plt.xlabel('Datetime')
plt.legend(["Training set", "Test set"])
plt.title("BLPH close price")
plt.show()

## Data Normalization
This method uses the MinMaxScaler, but a log return can sometimes be more useful. 

In [ ]:
# I have altered this so it works better w/ my data.
minmax = MinMaxScaler(feature_range=(0,1))
minmax_single = MinMaxScaler(feature_range=(0,1))

# no idea if the line below will work the way I need it to or not
train_scaled = np.concatenate([minmax.fit_transform(training_data.close_price.values)], axis=1)

train_scaled = pd.DataFrame(train_scaled, columns=training_data.columns)
train_scaled

In [ ]:
# validate min/max values
train_scaled.describe()

## Shaping data for LSTM

In [ ]:
train_scaled = train_scaled.values
n_lags = int(60) # not sure why hard 60
X_train = []
y_train = []
for i in range(n_lags, train_scaled.shape[0]):
    X_train.append(train_scaled[i-n_lags:i])
    y_train.append(train_scaled[i,0])

# check shape (again) before start training
'''shape_chk = []
for i in symbols:
    index = {}
    index["X_train"] = X_train.shape
    index["y_train"] = y_train.shape
    shape_chk.append(index)
pd.DataFrame(shape_chk)'''

## Build the LSTM network

In [ ]:
%%time

# LSTM architecture
regressor = tf.keras.Sequential()

# Layer 1, w/ dropout regularisation
regressor.add(tf.keras.layers.LSTM(units=20, return_sequences=True, input_shape=(X_train.shape[1]),4))
regressor.add(tf.keras.layers.Dropout(0.2))

# Layer 2
regressor.add(tf.keras.layers.LSTM(units=20, return_sequences))